In [1]:
from src.Base.Similarity.Compute_Similarity import SimilarityFunction
from src.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from src.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from src.KNN.ItemKNNCBFCFSimilarityHybridRecommender import ItemKNNCBFCFSimilarityHybridRecommender
from src.Base.Evaluation.Evaluator import EvaluatorHoldout
from src.Utils.load_URM import load_URM
from src.Utils.load_ICM import load_ICM
from operator import itemgetter
import numpy as np

URM_all = load_URM("in/data_train.csv")
ICM_all = load_ICM("in/data_ICM_title_abstract.csv")
from src.Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

In [2]:
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])


In [18]:
MAP_CBF=[]
MAP_CF=[]

for similarity in ['cosine','pearson','jaccard','tanimoto','adjusted','euclidean']:

    for topK in [10, 50, 100, 200, 500]:
        for shrink in [0, 10, 50, 100, 200, 500]:
            recommender = ItemKNNCFRecommender(URM_train)
            recommender.fit(topK=topK, shrink=shrink, similarity=similarity)

            result_dict, _ = evaluator_validation.evaluateRecommender(recommender)

            MAP_CF.append(('similarity={}'.format(similarity),'topK={}'.format(topK),'shrink={}'.format(shrink),result_dict[10]["MAP"]))

            print(MAP_CF[-1])

    for topK in [10, 50, 100, 200, 500]:
        for shrink in [0, 10, 50, 100, 200, 500]:
            recommender = ItemKNNCBFRecommender(URM_train=URM_train, ICM_train=ICM_all)
            recommender.fit(topK=topK, shrink=shrink, similarity=similarity)

            result_dict, _ = evaluator_validation.evaluateRecommender(recommender)

            MAP_CBF.append(('similarity={}'.format(similarity),'topK={}'.format(topK),'shrink={}'.format(shrink),result_dict[10]["MAP"]))
            print(MAP_CBF[-1])

ItemKNNCFRecommender: URM Detected 64 (0.81 %) cold users.
ItemKNNCFRecommender: URM Detected 2420 (9.32 %) cold items.
Similarity column 25975 ( 100 % ), 7418.95 column/sec, elapsed time 0.06 min
EvaluatorHoldout: Processed 5690 ( 100.00% ) in 3.19 sec. Users per second: 1783
('similarity=cosine', 'topK=10', 'shrink=0', 0.028207221188286516)
ItemKNNCFRecommender: URM Detected 64 (0.81 %) cold users.
ItemKNNCFRecommender: URM Detected 2420 (9.32 %) cold items.
Similarity column 25975 ( 100 % ), 6211.10 column/sec, elapsed time 0.07 min
EvaluatorHoldout: Processed 5690 ( 100.00% ) in 3.15 sec. Users per second: 1804
('similarity=cosine', 'topK=10', 'shrink=10', 0.04037655746246934)
ItemKNNCFRecommender: URM Detected 64 (0.81 %) cold users.
ItemKNNCFRecommender: URM Detected 2420 (9.32 %) cold items.
Similarity column 25975 ( 100 % ), 7521.29 column/sec, elapsed time 0.06 min
EvaluatorHoldout: Processed 5690 ( 100.00% ) in 3.09 sec. Users per second: 1844
('similarity=cosine', 'topK=10',

C:\Users\ALEXDRUSO\Desktop\RecSysChallenge2020\src\Base\Similarity\Compute_Similarity_Euclidean.py:170: RuntimeWarning: divide by zero encountered in true_divide
  item_distance /= sumOfSquared[columnIndex] * sumOfSquared
C:\Users\ALEXDRUSO\Desktop\RecSysChallenge2020\src\Base\Similarity\Compute_Similarity_Euclidean.py:170: RuntimeWarning: invalid value encountered in true_divide
  item_distance /= sumOfSquared[columnIndex] * sumOfSquared


TypeError: 'coo_matrix' object is not subscriptable

In [19]:
max_tuple = max(MAP_CF, key=itemgetter(3))
print('The best combination for the ICF recommender is ({},{},{}), with a MAP = {}'.format(max_tuple[0],max_tuple[1], max_tuple[2], max_tuple[3]))

max_tuple = max(MAP_CBF, key=itemgetter(3))
print('The best combination for the CBF recommender is ({},{},{}), with a MAP = {}'.format(max_tuple[0],max_tuple[1], max_tuple[2], max_tuple[3]))



The best combination for the ICF recommender is (similarity=cosine,topK=200,shrink=50), with a MAP = 0.048583514880924626
The best combination for the CBF recommender is (similarity=jaccard,topK=100,shrink=10), with a MAP = 0.03307492333488752


In [3]:
recommender = ItemKNNCBFCFSimilarityHybridRecommender(
    URM_train=URM_train,
    ICM_train=ICM_all,
    topK_knncf=200,
    shrink_knncf=50,
    similarity_knncf='cosine',
    topK_knncbf=100,
    shrink_knncbf=10,
    similarity_knncbf='jaccard'
)

ItemKNNCFRecommender: URM Detected 77 (0.97 %) cold users.
ItemKNNCFRecommender: URM Detected 2422 (9.32 %) cold items.
Similarity column 25975 ( 100 % ), 5867.90 column/sec, elapsed time 0.07 min
ItemKNNCBFRecommender: URM Detected 77 (0.97 %) cold users.
ItemKNNCBFRecommender: URM Detected 2422 (9.32 %) cold items.
Similarity column 25975 ( 100 % ), 4504.42 column/sec, elapsed time 0.10 min
 ItemKNNCBFCFSimilarityHybridRecommender: URM Detected 77 (0.97 %) cold users.
 ItemKNNCBFCFSimilarityHybridRecommender: URM Detected 2422 (9.32 %) cold items.


In [4]:
MAP=[]

for topK in [10, 50, 100, 200, 500, 600]:
    for alpha in np.linspace(start=0.0,stop=1.0,num=15):
        recommender.fit(topK=topK,alpha=alpha)

        result_dict, _ = evaluator_validation.evaluateRecommender(recommender)

        MAP.append({'topK':topK,'alpha':alpha,'MAP':result_dict[10]["MAP"]})
        print(MAP[-1])

EvaluatorHoldout: Processed 5691 ( 100.00% ) in 3.06 sec. Users per second: 1860
{'topK': 10, 'alpha': 0.0, 'MAP': 0.03256112668161557}
EvaluatorHoldout: Processed 5691 ( 100.00% ) in 3.07 sec. Users per second: 1855
{'topK': 10, 'alpha': 0.07142857142857142, 'MAP': 0.033523950569248666}
EvaluatorHoldout: Processed 5691 ( 100.00% ) in 3.07 sec. Users per second: 1853
{'topK': 10, 'alpha': 0.14285714285714285, 'MAP': 0.03488429794202378}
EvaluatorHoldout: Processed 5691 ( 100.00% ) in 3.08 sec. Users per second: 1849
{'topK': 10, 'alpha': 0.21428571428571427, 'MAP': 0.03668602301844739}
EvaluatorHoldout: Processed 5691 ( 100.00% ) in 3.06 sec. Users per second: 1857
{'topK': 10, 'alpha': 0.2857142857142857, 'MAP': 0.03828328212487927}
EvaluatorHoldout: Processed 5691 ( 100.00% ) in 2.98 sec. Users per second: 1908
{'topK': 10, 'alpha': 0.3571428571428571, 'MAP': 0.0408062994765839}
EvaluatorHoldout: Processed 5691 ( 100.00% ) in 3.03 sec. Users per second: 1880
{'topK': 10, 'alpha': 0.4

In [10]:
max_tuple = max(MAP, key= lambda x: x['MAP'])
print('The best combination for the hybrid recommender is ({},{}), with a MAP = {}'.format(max_tuple['topK'],max_tuple['alpha'], max_tuple['MAP']))


The best combination for the hybrid recommender is (500,0.7857142857142857), with a MAP = 0.06377184433422467
